In [1]:
import pymysql

In [2]:
def connect():
    con = pymysql.connect(
        host = 'localhost',
        port = 3306,
        user = 'root',
        password = 'gkdks120',
        db = 'ML_DB',
        charset = 'utf8'
    )
    print('1. 연결성공')
    cur = con.cursor()
    print('2. 커서 객체 획득')
    return cur,con

In [3]:
cur2, con = connect()
cur2

1. 연결성공
2. 커서 객체 획득


In [4]:
# select라 con 안써줌.. con은 commit용 (데이터가 변경될때)
def sql_execute(cur2, data):
    sql = '''
    select o2.orderid, m.name, o2.productid, o2.totalprice
    from orderlist o2
    inner join `member` m
    on o2.memberid = m.id and m.id = %s
    order by productid desc
    '''
    result = cur2.execute(sql,data)
    return result

In [5]:
sql_result = sql_execute(cur2, 'park')
sql_result

5

In [6]:
## 1. 누가 어떤 물건을 얼마나 샀는가?

In [7]:
who = input('누가?? >> ')

누가?? >> park


In [8]:
sql_result2 = sql_execute(cur2,who)
sql_result2

5

In [9]:
def out_data(cur2):
    rows = cur2.fetchall()
    return rows

In [10]:
result_rows = out_data(cur2)
result_rows

((2, 'park', '101', 6000),
 (1, 'park', '100', 5000),
 (3, 'park', '100', 10000),
 (4, 'park', '100', 3000),
 (8, 'park', '100', 1000))

In [11]:
def db_close(con):
    con.close()

In [12]:
db_close(con)

In [13]:
## 제일 높은 금액은?

In [14]:
[x[3] for x in result_rows]

[6000, 5000, 10000, 3000, 1000]

In [15]:
max([x[3] for x in result_rows])

10000

In [16]:
## 제일 많이 산 물건의 id는?

In [17]:
import pandas as pd

In [18]:
result_df = pd.DataFrame(data=result_rows,
                        columns=['주문id','name','제품id','주문총가격'])

In [19]:
result_df

,주문id,name,제품id,주문총가격
0,2,park,101,6000
1,1,park,100,5000
2,3,park,100,10000
3,4,park,100,3000
4,8,park,100,1000


In [20]:
[x for x in result_df['제품id']]

['101', '100', '100', '100', '100']

In [21]:
from collections import Counter

In [22]:
counter_id = Counter(result_df['제품id'])

In [23]:
counter_id

Counter({'101': 1, '100': 4})

In [24]:
max_id = counter_id.most_common(n=1)
print(max_id)
print(max_id[0][0])
max_id_result = max_id[0][0]
max_id_result

[('100', 4)]
100


'100'

In [25]:
def sql_execute2(cur2,data):
    sql = '''
    select * from product
    where id = %s
    '''
    result2 = cur2.execute(sql,data)
    return result2

In [26]:
cur2, con = connect()
cur2

1. 연결성공
2. 커서 객체 획득


In [27]:
sql_result2 = sql_execute2(cur2, max_id_result)
sql_result2

1

In [28]:
result_rows = out_data(cur2)
result_rows[0]

('100', 'java', 'fun java', '1000')

In [29]:
print('당신에게 추천하고 싶은 물건은 다음과 같습니다.')
print('------------------------------------')
print('물건의 이름은 ',result_rows[0][1])
print('물건의 내용은', result_rows[0][2])
print('물건의 가격은', result_rows[0][3])

당신에게 추천하고 싶은 물건은 다음과 같습니다.
------------------------------------
물건의 이름은  java
물건의 내용은 fun java
물건의 가격은 1000


In [30]:
# select라 con 안써줌.. con은 commit용 (데이터가 변경될때)
def sql_execute3(con,cur2,data):
    sql = '''
    insert into orderlist values (null,%s,%s,%s)
    '''
    result2 = cur2.execute(sql,data)
    con.commit()
    return result2

In [31]:
order_question = input('구매하시겠습니까?(yes/no)')
if order_question == 'yes':
    print('---------구매를 진행합니다.----------')
    your_id = input('당신의 id는 ')
    data_list = [your_id, result_rows[0][0], int(result_rows[0][3])]
    result3 = sql_execute3(con,cur2,data_list)
    if result3 == 1:
        print('---구매가 완료되었습니다.')
    else:
        print('---구매에 문제가 발생했습니다.')

구매하시겠습니까?(yes/no)no


In [32]:
con.close()

In [33]:
# 추천을 하고자 하는 컨텐츠기반 룰을 정하고,
### 고객이 쓴 게시글 제목기반으로 상품 추천

# 물건을 3개 이상 추천하는 추천시스템을 완성하시오
### counter_id.most_common(n=3)

# db의 데이터를 임의로 추가할 수 있음

In [55]:
cur3, con = connect()

1. 연결성공
2. 커서 객체 획득


In [56]:
def yb_execute(cur, data):
    sql = '''
    select b.title, p.price, p.id ,b.writer
    from board b
    inner join product p
    on b.title = p.name and b.writer = %s
    '''
    result = cur.execute(sql,data)
    return result

In [57]:
who = input('누가?? >> ')

누가?? >> apple


In [58]:
sql_result3 = yb_execute(cur3,who)
sql_result3

4

In [59]:
result_rows = out_data(cur3)
result_rows2 = list(set(result_rows))
result_rows2

[('java', '1000', '100', 'apple'),
 ('spring', '3000', '102', 'apple'),
 ('android', '4000', '103', 'apple')]

In [60]:
result_df = pd.DataFrame(data=result_rows,
                        columns=['title','price','p_id','writer'])

In [61]:
result_df

,title,price,p_id,writer
0,java,1000,100,apple
1,java,1000,100,apple
2,spring,3000,102,apple
3,android,4000,103,apple


In [62]:
counter_id = Counter(result_df['title'])
counter_id

Counter({'java': 2, 'spring': 1, 'android': 1})

In [63]:
max_id = counter_id.most_common(n=3)
print(max_id)
print(max_id[0][0])
max_id_result = max_id[0][0]
max_id_result

[('java', 2), ('spring', 1), ('android', 1)]
java


'java'

In [64]:
cur3, con = connect()
cur3

1. 연결성공
2. 커서 객체 획득


In [65]:
print('당신에게 추천하고 싶은 물건은 다음과 같습니다.')
print('------------------------------------')
for i in range(3):    
    print('물건의 이름은',result_rows2[i][0], "물건의 고유번호는",result_df['p_id'][result_df["title"] == result_rows2[i][0]].values[0] )
    print('가격은', result_rows2[i][1])
    print('당신이 관련 게시글을 쓴 횟수',result_df['title'][result_df["title"] == result_rows2[i][0]].value_counts().values[0], end="\n\n" )

당신에게 추천하고 싶은 물건은 다음과 같습니다.
------------------------------------
물건의 이름은 java 물건의 고유번호는 100
가격은 1000
당신이 관련 게시글을 쓴 횟수 2

물건의 이름은 spring 물건의 고유번호는 102
가격은 3000
당신이 관련 게시글을 쓴 횟수 1

물건의 이름은 android 물건의 고유번호는 103
가격은 4000
당신이 관련 게시글을 쓴 횟수 1



In [66]:
purchase = input("사고싶은 물건의 고유번호를 입력하세요: ")

사고싶은 물건의 고유번호를 입력하세요: 100


In [67]:
# select라 con 안써줌.. con은 commit용 (데이터가 변경될때)
def sql_execute3(con,cur,data):
    sql = '''
    insert into orderlist values (null,%s,%s,%s)
    '''
    result2 = cur.execute(sql,data)
    con.commit()
    return result2

In [68]:
result_df['price'][result_df['p_id']==purchase].values[0]

'1000'

In [70]:
order_question = input('구매하시겠습니까?(yes/no)')
if order_question == 'yes':
    print('---------구매를 진행합니다.----------')
    your_id = input('당신의 id는 ')
    data_list = [your_id, purchase, int(result_df['price'][result_df['p_id']==purchase].values[0])]
    result3 = sql_execute3(con,cur3,data_list)
    if result3 == 1:
        print('---구매가 완료되었습니다.')
    else:
        print('---구매에 문제가 발생했습니다.')

구매하시겠습니까?(yes/no)yes
---------구매를 진행합니다.----------
당신의 id는 apple
---구매가 완료되었습니다.


In [ ]:
con.close()